In [2]:
pip install requests bs4 pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests #send http request
import pandas as pd #data manipulation -send data to db
from bs4 import BeautifulSoup #htmlparser

In [ ]:
function = scrape_pages(start_page, end_page)

base_url = 'https://www.buyrentkenya.com/houses-for-sale'

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

In [ ]:
def scrape_pages(start_page, end_page)
                 base_url = 'https://www.buyrentkenya.com/houses-for-sale'
                 header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
                           }
                 